In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def load(image_path):
    return cv2.imread(image_path, 0).astype(float) # 0 reads as grayscale (color would be 1)


# Make matplotlib figures appear inline in the
# notebook rather than in a new window
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

def display(img, title=None):
    # Show image
    plt.figure(figsize = (5,5))
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

The goal of this project is to detect rocket panels and predict the distance to them. The targets will be found in images with names beginning with "RocketPanelStraightDark" (16" and higher so the vision targets are visible) by converting them to grayscale and running a brightness threshold. The targets will then be found using contour mapping. The distance to the targets can be determined as the camera type and therefore vertical FOV is known (except these test images are at 4:3 instead of 16:9 so the FOV numbers I have are probably not perfectly accurate) as is the physical height of the targets which allows for the distance to be calculated. In order for a truly accurate distance to be found, the camera distortion must be known, which is not the case here.

This process is commonly applied in FRC, although our team has been using homographies for the last two years which allows us to better identify targets and get more accurate information about the robot's position from them.

In [13]:
# FOV @ 4:3 aspect ratio (degrees)
VERT_FOV = 41.1
HORIZ_FOX = 54.8
TAPE_HEIGHT = 5.5 * np.cos(np.deg2rad(14.5)) + 2 * np.sin(np.deg2rad(14.5)) # inches. Expected bbox height
# each tape is 5.5 inches long, 2 wide, and at a 14.5 degree angle from the horizontal
IMG_HEIGHT = 240
IMG_WIDTH = 320

# grayscale -- unused because we can do it while loading
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# binary threshold
def threshold(img):
    return cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1] # [0] is just 127

# find contours
def contours(thresh):
    contours, hierarchy = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# filter for vision target contours
def filter_contours(contours):
    return sorted(contours, key=cv2.contourArea)[-2:]

class Bbox:
    def __init__(self, x, y, w, h): # x and y are ctr, w and h are self explanatory
        self.x = x
        self.y = y
        self.w = w
        self.h = h

# get the bounding box of a contour
def get_bbox(contour):
    x, y, w, h = cv2.boundingRect(contour)
    return Bbox((x + w) / 2, (y + h) / 2, w, h)

def get_dist(bbox):
    target_vert_fov = bbox.h * np.deg2rad(VERT_FOV) / IMG_HEIGHT # this is where a linear mapping of pixels to degrees is assumed
    dist_px = bbox.h / np.tan(target_vert_fov) # distance in pixels. Assumption: right triangle
    px2in = bbox.h / TAPE_HEIGHT
    return dist_px / px2in # inches
     

def dist_from_img(img_str):
    start_img = load(img_str)
    thresh_img = threshold(start_img)    
    contours_array = contours(thresh_img)
    filtered_contours = filter_contours(contours_array)
    bboxes = [get_bbox(contour) for contour in filtered_contours]
    dists = [get_dist(bbox) for bbox in bboxes]
    return np.mean(dists)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
test_dists = [16, 24, 36, 48, 60, 72, 96]
test_strs = ["2019VisionImages/RocketPanelStraightDark" + str(x) + "in.jpg" for x in test_dists]
for i, test_str in enumerate(test_strs):
    print("ground truth: " + str(test_dists[i]) + ", estimation: " + str(dist_from_img(test_str)))

ground truth: 16, estimation: 16.134966993643857
ground truth: 24, estimation: 24.538008980725824
ground truth: 36, estimation: 36.82357339855257
ground truth: 48, estimation: 49.12234433844229
ground truth: 60, estimation: 60.723044690088685
ground truth: 72, estimation: 73.4226753374948
ground truth: 96, estimation: 175.8835389255266


These results pretty good, especially considering the assumptions made and the fact that these FOV's are kind of sketchy and the actual pixels to degrees mapping is unknown. Additionally, I believe that the ground truth distances are not to the targets but rather to part of the structures that they are attached to, which makes it possible that the first few results are more accurate than they seem.